TACOTRON

De/Encoder pre net

How it's implemented on TF:

    def prenet(inputs, is_training=True, scope="prenet", reuse=None):
        with tf.variable_scope(scope, reuse=reuse):
            outputs = tf.layers.dense(inputs, units=hp.embed_size, activation=tf.nn.relu, name="dense1")
            outputs = tf.nn.dropout(outputs, keep_prob=.5 if is_training==True else 1., name="dropout1")
            outputs = tf.layers.dense(outputs, units=hp.embed_size//2, activation=tf.nn.relu, name="dense2")
            outputs = tf.nn.dropout(outputs, keep_prob=.5 if is_training==True else 1., name="dropout2") 
        return outputs # (N, T, num_units/2)

In [1]:
from __future__ import print_function
import mxnet as mx
import numpy as np
from mxnet import nd, autograd
from mxnet import gluon
from IPython.display import clear_output

In [2]:
ctx= mx.cpu()
num_hidden = 256
K=16
emb_size=256;

data = mx.sym.Variable('data')
emb = mx.sym.Embedding(data=data, input_dim=100, output_dim=emb_size)


In [3]:

def prenet():
    prenet = gluon.nn.Sequential() #consider to use HybridSequential
    with prenet.name_scope():
        prenet.add(gluon.nn.Dense(num_hidden,activation="relu"))
        prenet.add(gluon.nn.Dropout(0.5)) #how to use the condition "if train" same as TF?
        prenet.add(gluon.nn.Dense(num_hidden//2,activation="relu"))
        prenet.add(gluon.nn.Dropout(0.5))
    prenet.collect_params().initialize()
    #prenet.initialize() seems the same
    return prenet

In [4]:
a = prenet()
print(a)

Sequential(
  (0): Dense(256, Activation(relu))
  (1): Dropout(p = 0.5)
  (2): Dense(128, Activation(relu))
  (3): Dropout(p = 0.5)
)


In [22]:
#banco di filtri convolutivi. Vengono creati K filtri con kernel 1D di dimensione:k 
def conv1dBank(conv_input):
    conv = mx.sym.Activation(data=mx.sym.Convolution(data=conv_input, kernel=(1,1), num_filter=emb_size//2), act_type='relu')
    for k in range(2, K+1):
        convi = mx.sym.Convolution(data=conv_input, kernel=(k,1), num_filter=emb_size//2)
        (convi, mean, var) = mx.sym.BatchNorm(data=convi, output_mean_var=True) #TOSTUDY: batch norm?
        convi = mx.sym.Activation(data=convi, act_type='relu')
        conv = mx.symbol.concat(conv,convi)
        #pooled_outputs.append(pooli)
    return conv

convbank=conv1dBank(emb)
#mx.viz.plot_network(convbank)
#se si usa infer_shape su convbank dando la dimensione dell'input, viene dedotta la shape appunto 

In [17]:
#highway #TOSTUDY
def highway(data):
    H= mx.symbol.Activation(data=mx.symbol.FullyConnected(data=data, num_hidden=emb_size//2), act_type="relu")
    T= mx.symbol.Activation(data=mx.symbol.FullyConnected(data=data, num_hidden=emb_size//2, bias=mx.sym.Variable('bias') ), act_type="sigmoid")
    return  H * T + data * (1.0 - T)


In [34]:


#CBHG
def CBHG(data):
    bank = conv1dBank(data)
    poold_bank = mx.sym.Pooling(data=bank, pool_type='max', kernel=(2, 1), stride=(1,1)) #TOSTUDY: stride?

    proj1 = mx.sym.Convolution(data=poold_bank, kernel=(3,1), num_filter=emb_size//2)
    (proj1, proj1_mean, proj1_var) = mx.sym.BatchNorm(data=proj1, output_mean_var=True) #TOSTUDY: does the symbol encapsule the mean and var too?
    proj1 = mx.sym.Activation(data=proj1, act_type='relu')#Is it ok to declare/use again the same variable?  

    proj2 = mx.sym.Convolution(proj1, kernel=(3,1), num_filter=emb_size//2)
    (proj2, proj2_mean, proj2_var) = mx.sym.BatchNorm(data=proj2, output_mean_var=True)
    residual= proj2 + emb

    hw = highway(residual)
    mx.viz.plot_network(CBHG(emb) )
    net = gluon.nn.Sequential()
    with net.name_scope():
        for i in range(0,4):
            #net.add(gluon.nn.Dense(100))
            print(i)
        
    return net
CBHG(emb)


RecursionError: maximum recursion depth exceeded in comparison